In [1]:
import os
import whisper
import openai
import requests_html
import dataset
import pprint

from dotenv import load_dotenv
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [2]:
navegador = Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
navegador.maximize_window()

In [4]:
navegador.get("https://pyvideo.org/index.html")

In [5]:
links = navegador.find_elements(By.XPATH, '//article/a')

In [6]:
for link in links:
    print(link.get_attribute('href'))

https://pyvideo.org/chipy/Exploring_the_Python_Run_Time_Environment.html
https://pyvideo.org/chipy/JSON_Web_Tokens_for_Fun_and_Profit.html
https://pyvideo.org/chipy/Ellipses_and_Arcane_Syntax.html
https://pyvideo.org/chipy/Micropython_gpio.html
https://pyvideo.org/chipy/Learning_Sprint_An_Experiment.html
https://pyvideo.org/pytexas-2023/day-2-keynote-full-stack-python.html


In [7]:
links[0].click()

In [8]:
links_video = navegador.find_elements(By.LINK_TEXT, 'ARCHIVE')

In [9]:
for link_video in links_video:
    print(link_video.get_attribute('href'))

https://pyvideo.org/chipy/Exploring_the_Python_Run_Time_Environment.html#archive
https://archive.org/details/June_23-Exploring_the_Python_Run_Time_Environment/Exploring_the_Python_Run_Time_Environment.mp4


In [10]:
video_url = links_video[1].get_attribute('href')
video_url

'https://archive.org/details/June_23-Exploring_the_Python_Run_Time_Environment/Exploring_the_Python_Run_Time_Environment.mp4'

In [11]:
sessao = requests_html.HTMLSession()

In [12]:
resposta = sessao.get(video_url)

In [13]:
video_download = resposta.html.xpath('//a[contains(text(), "MPEG4")]/@href', first=True)
video_download

'/download/June_23-Exploring_the_Python_Run_Time_Environment/Exploring_the_Python_Run_Time_Environment.mp4'

In [14]:
resposta = sessao.get(f'https://archive.org{video_download}')
with open('video.mp4', 'wb') as arquivo:
    arquivo.write(resposta.content)

In [15]:
modelo = whisper.load_model("small")

In [48]:
ffmpeg_extract_subclip("video.mp4", 0, 120, targetname="video_cortado.mp4")

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [16]:
transcricao = modelo.transcribe('video_cortado.mp4')

/home/gileno/workspace/nxtweb/venv/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [17]:
pprint.pprint(transcricao)

{'language': 'en',
 'segments': [{'avg_logprob': -0.18204337559389264,
               'compression_ratio': 1.5848214285714286,
               'end': 5.5600000000000005,
               'id': 0,
               'no_speech_prob': 0.05237644910812378,
               'seek': 0,
               'start': 0.0,
               'temperature': 0.0,
               'text': " All right, so I guess I'm going to start with my "
                       'motivation for the talk.',
               'tokens': [50364,
                          1057,
                          558,
                          11,
                          370,
                          286,
                          2041,
                          286,
                          478,
                          516,
                          281,
                          722,
                          365,
                          452,
                          12335,
                          337,
                          264,
    

In [18]:
pprint.pprint(transcricao['text'])

(" All right, so I guess I'm going to start with my motivation for the talk. "
 'So one of the things that always kind of confused me about Python is you '
 'write something in Python and then you click run and then the computer does '
 'some magic and it pops out an answer for you. And when I was looking through '
 'the C-Python interpreter, the core system that actually runs Python, the '
 'thing that conceptually gave me the hardest challenge of understanding was '
 "the runtime environment. So I'm going to start with a brief overview of what "
 "the C-Python interpreter is and what it does. So let's say I'm a clever, "
 'lazy programmer who was given the assignment that they need a program that '
 'is completely secure and bug-free. So I was like, if I just delete '
 "everything, then it's perfectly bug-free and secure. So how C-Python works "
 "is that there's basically two parts. There's the compiler that converts any "
 "Python code into a byte code and then there's the runtime,

In [19]:
load_dotenv()

True

In [20]:
openai.api_key = os.environ['OPENAI_KEY']

In [21]:
mensagens = []
mensagens.append(
    {"role": "user", "content": f"traduza o seguinte de inglês para português: {transcricao['text']}"},
)
chat = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=mensagens
)
chat['choices'][0]['message']['content']

'Tudo bem, então eu acho que vou começar com minha motivação para a palestra. Então, uma das coisas que sempre me deixou confuso sobre o Python é que você escreve algo em Python, clica em executar e o computador faz alguma mágica e retorna uma resposta para você. E quando estava olhando o interpretador C-Python, o sistema central que executa o Python, a coisa que conceitualmente me trouxe o maior desafio de compreensão foi o ambiente de execução. Então vou começar com uma breve visão geral do que é o interpretador C-Python e o que ele faz. Então vamos supor que sou um programador esperto e preguiçoso que recebeu a tarefa de criar um programa que seja completamente seguro e livre de bugs. Então pensei, se eu simplesmente deletar tudo, ele será perfeitamente livre de bugs e seguro. A forma como o C-Python funciona é que existem basicamente duas partes. Há o compilador que converte qualquer código Python em um código de bytes e, em seguida, há o tempo de execução, que é a coisa que realme